In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.tabular import *

In [3]:
from fastai import *
from fastai.tabular import *
from fastai.utils import *

In [4]:
import fastai
print(fastai.__version__)

1.0.39.dev0


In [5]:
torch.cuda.set_device(1)

In [6]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [7]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [8]:
%xmode Plain

Exception reporting mode: Plain


# Rossmann

## Data preparation

To create the feature-engineered train_clean and test_clean from the Kaggle competition data, run `rossman_data_clean.ipynb`. One important step that deals with time series is this:

```python
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)
```

In [9]:
path = Path('../data/rossmann/')
train_df = pd.read_pickle(path/'train_clean')

In [10]:
train_df.head().T

,0,1,2,3,4
index,0,1,2,3,4
Store,1,2,3,4,5
DayOfWeek,5,5,5,5,5
Date,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00
Sales,5263,6064,8314,13995,4822
Customers,555,625,821,1498,559
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,False,False,False,False,False
SchoolHoliday,1,1,1,1,1


In [11]:
n = len(train_df); n

844338

### Preparing full data set

In [12]:
train_df = pd.read_pickle(path/'train_clean')
test_df = pd.read_pickle(path/'test_clean')

In [13]:
len(train_df),len(test_df)

(844338, 41088)

In [14]:
procs=[FillMissing, Categorify, Normalize]

In [39]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw']

#dropped 'CompetitionDistance', 'CloudCover'
cont_vars = ['Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

In [40]:
dep_var = 'Sales'
df = train_df[cat_vars + cont_vars + [dep_var,'Date']].copy()

In [41]:
test_df['Date'].min(), test_df['Date'].max()

(Timestamp('2015-08-01 00:00:00'), Timestamp('2015-09-17 00:00:00'))

In [42]:
cut = train_df['Date'][(train_df['Date'] == train_df['Date'][len(test_df)])].index.max()
cut

41395

In [43]:
valid_idx = range(cut)

In [44]:
val_df = df.iloc[valid_idx]

In [45]:
len(val_df)

41395

In [22]:
#dif_df=df.where(val_df.values!=df.values)

In [23]:
#dif_df.head()

In [24]:
#check validation dataframe for data not in train df

print('checking val vs df classes')
col_diffs={}
for col in df:
    #new set with elements in val_df but not in df
    diffs = set(val_df[col]).difference(set(df[col]))
    print(f'col: {col}, diffs: {len(diffs)}')
    if len(diffs)>0:
        col_diffs[col]=diffs


checking val vs df classes
col: Store, diffs: 0
col: DayOfWeek, diffs: 0
col: Year, diffs: 0
col: Month, diffs: 0
col: Day, diffs: 0
col: StateHoliday, diffs: 0
col: CompetitionMonthsOpen, diffs: 0
col: Promo2Weeks, diffs: 0
col: StoreType, diffs: 0
col: Assortment, diffs: 0
col: PromoInterval, diffs: 0
col: CompetitionOpenSinceYear, diffs: 0
col: Promo2SinceYear, diffs: 0
col: State, diffs: 0
col: Week, diffs: 0
col: Events, diffs: 0
col: Promo_fw, diffs: 0
col: Promo_bw, diffs: 0
col: StateHoliday_fw, diffs: 0
col: StateHoliday_bw, diffs: 0
col: SchoolHoliday_fw, diffs: 0
col: SchoolHoliday_bw, diffs: 0
col: CompetitionDistance, diffs: 111
col: Max_TemperatureC, diffs: 0
col: Mean_TemperatureC, diffs: 0
col: Min_TemperatureC, diffs: 0
col: Max_Humidity, diffs: 0
col: Mean_Humidity, diffs: 0
col: Min_Humidity, diffs: 0
col: Max_Wind_SpeedKm_h, diffs: 0
col: Mean_Wind_SpeedKm_h, diffs: 0
col: CloudCover, diffs: 3869
col: trend, diffs: 0
col: trend_DE, diffs: 0
col: AfterStateHoliday, d

In [34]:
cols_to_drop=list(col_diffs.keys())
cols_to_drop

['CompetitionDistance', 'CloudCover']

In [35]:
#Drop df colls with val/train df differences
df.drop(columns=cols_to_drop,inplace=True)
test_df.drop(columns=cols_to_drop,inplace=True)

In [36]:
df[dep_var].head()

0     5263
1     6064
2     8314
3    13995
4     4822
Name: Sales, dtype: int64

In [37]:
def unique_deps(x:Series)->List:
    od = OrderedDict.fromkeys(x)
    res = list(OrderedDict.fromkeys(x).keys())
    res.sort()
    return res, od

In [38]:
classes, od =unique_deps(df[dep_var].values)

In [46]:
data = TabularDataBunch.from_df(path, df=df, dep_var=dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_vars, cont_names=cont_vars)
#on learn.fit get
#AssertionError: Expected same numbers of elements in pred & targ

--label_from_df() cols: Sales
--label_from_df() cols: Sales


Exception: Your validation data contains a label that isn't present in the training set, please fix your data.

In [ ]:
%debug

In [ ]:
#data = (TabularList.from_df(df, path=path, cat_names=cat_vars, cont_names=cont_vars, procs=procs, test_df=test_df)
#                   .split_by_idx(valid_idx)
#                   .label_from_df(cols=dep_var, label_cls=FloatList, log=True)
#                   .databunch())

In [ ]:
#doc(FloatList)

## Model

In [ ]:
max_log_y = np.log(np.max(train_df['Sales'])*1.2)
y_range = torch.tensor([0, max_log_y], device=defaults.device)

In [ ]:
learn = tabular_learner(data, layers=[1000,500], ps=[0.001,0.01], emb_drop=0.04, 
                        y_range=y_range, metrics=exp_rmspe)

In [ ]:
learn.model

In [ ]:
len(data.train_ds.cont_names)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
#minimum at 1e-2

In [ ]:
learn.fit_one_cycle(3, 1e-3, wd=0.2)

using:
data = TabularDataBunch.from_df(path, df=df, dep_var=dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_vars, cont_names=cont_vars, classes=classes, test_df=test_df)

<pre>
/mnt/963GB/Data/Python/Courses/fastai/fastai/fastai/callback.py in on_batch_end(self=<fastai.callback.AverageMetric object>, last_output=tensor([[0.1735, 1.0371, 0.1735,  ..., 0.1735, 0...0.1494, 0.1494, 0.1494]],
       device='cuda:1'), last_target=[tensor([ 4406,  5207,  7457, 13138,  3965,  4794...     4715,  6638, 10668, 12394], device='cuda:1')], **kwargs={'epoch': 0, 'iteration': 12545, 'last_input': [tensor([[ 1,  5,  3,  ...,  6,  1,  1],
        ...[64,  5,  3,  ...,  6,  1,  1]], device='cuda:1'), tensor([[-0.5358,  1.0954,  0.8912,  ...,  1.184...1848,  1.1119,  2.0852]],
       device='cuda:1')], 'last_loss': tensor(10.0581), 'metrics': [<function exp_rmspe>], 'n_epochs': 3, 'num_batch': 12545, 'pbar': <fastprogress.fastprogress.NBMasterBar object>, 'smooth_loss': tensor(9.8903), 'train': False})
    272         if not is_listy(last_target): last_target=[last_target]
    273         self.count += last_target[0].size(0)
--> 274         self.val += last_target[0].size(0) * self.func(last_output, *last_target).detach().cpu()
        self.val = 0.0
        last_target.size = undefined
        self.func = <function exp_rmspe at 0x7fa2f3d13e18>
        last_output = tensor([[0.1735, 1.0371, 0.1735,  ..., 0.1735, 0.1735, 0.1735],
        [0.1522, 0.7655, 0.1522,  ..., 0.1522, 0.1522, 0.1522],
        [0.1632, 0.9980, 0.1632,  ..., 0.1632, 0.1632, 0.1632],
        ...,
        [0.1532, 0.8678, 0.1532,  ..., 0.1532, 0.1532, 0.1532],
        [0.1598, 0.9645, 0.1598,  ..., 0.1598, 0.1598, 0.1598],
        [0.1494, 0.8271, 0.1494,  ..., 0.1494, 0.1494, 0.1494]],
       device='cuda:1')
        last_target.detach.cpu = undefined
    275 
    276     def on_epoch_end(self, **kwargs):

/mnt/963GB/Data/Python/Courses/fastai/fastai/fastai/metrics.py in exp_rmspe(pred=tensor([[0.1735, 1.0371, 0.1735,  ..., 0.1735, 0...0.1494, 0.1494, 0.1494]],
       device='cuda:1'), targ=tensor([ 4406,  5207,  7457, 13138,  3965,  4794...     4715,  6638, 10668, 12394], device='cuda:1'))
     45 def exp_rmspe(pred:FloatTensor, targ:FloatTensor)->Rank0Tensor:
     46     "Exp RMSE between `pred` and `targ`."
---> 47     assert pred.numel() == targ.numel(), "Expected same numbers of elements in pred & targ"
        pred.numel = <built-in method numel of Tensor object at 0x7fa2b996f750>
        targ.numel = <built-in method numel of Tensor object at 0x7fa2b996f2d0>
     48     if len(pred.shape)==2: pred=pred.squeeze(1)
     49     pred, targ = torch.exp(pred), torch.exp(targ)

AssertionError: Expected same numbers of elements in pred & targ
</pre>

ipdb> pred.shape
torch.Size([64, 21733])
ipdb> targ.shape
torch.Size([64])

In [ ]:
%debug

<pre>

data = TabularDataBunch.from_df(path, df=df, dep_var=dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_vars, cont_names=cont_vars, classes=classes, test_df=test_df)


AssertionError                            Traceback (most recent call last)
<ipython-input-57-be485bf5210e> in <module>
----> 1 learn.fit_one_cycle(1, 3e-2, wd=0.2)

/mnt/963GB/Data/Python/Courses/fastai/fastai/fastai/train.py in fit_one_cycle(learn, cyc_len, max_lr, moms, div_factor, pct_start, wd, callbacks, **kwargs)
     20     callbacks.append(OneCycleScheduler(learn, max_lr, moms=moms, div_factor=div_factor,
     21                                         pct_start=pct_start, **kwargs))
---> 22     learn.fit(cyc_len, max_lr, wd=wd, callbacks=callbacks)
     23 
     24 def lr_find(learn:Learner, start_lr:Floats=1e-7, end_lr:Floats=10, num_it:int=100, stop_div:bool=True, **kwargs:Any):

/mnt/963GB/Data/Python/Courses/fastai/fastai/fastai/basic_train.py in fit(self, epochs, lr, wd, callbacks)
    164         callbacks = [cb(self) for cb in self.callback_fns] + listify(callbacks)
    165         fit(epochs, self.model, self.loss_func, opt=self.opt, data=self.data, metrics=self.metrics,
--> 166             callbacks=self.callbacks+callbacks)
    167 
    168     def create_opt(self, lr:Floats, wd:Floats=0.)->None:

/mnt/963GB/Data/Python/Courses/fastai/fastai/fastai/basic_train.py in fit(epochs, model, loss_func, opt, data, callbacks, metrics)
     92     except Exception as e:
     93         exception = e
---> 94         raise e
     95     finally: cb_handler.on_train_end(exception)
     96 

/mnt/963GB/Data/Python/Courses/fastai/fastai/fastai/basic_train.py in fit(epochs, model, loss_func, opt, data, callbacks, metrics)
     87             if not data.empty_val:
     88                 val_loss = validate(model, data.valid_dl, loss_func=loss_func,
---> 89                                        cb_handler=cb_handler, pbar=pbar)
     90             else: val_loss=None
     91             if cb_handler.on_epoch_end(val_loss): break

/mnt/963GB/Data/Python/Courses/fastai/fastai/fastai/basic_train.py in validate(model, dl, loss_func, cb_handler, pbar, average, n_batch)
     52             if not is_listy(yb): yb = [yb]
     53             nums.append(yb[0].shape[0])
---> 54             if cb_handler and cb_handler.on_batch_end(val_losses[-1]): break
     55             if n_batch and (len(nums)>=n_batch): break
     56         nums = np.array(nums, dtype=np.float32)

/mnt/963GB/Data/Python/Courses/fastai/fastai/fastai/callback.py in on_batch_end(self, loss)
    237         "Handle end of processing one batch with `loss`."
    238         self.state_dict['last_loss'] = loss
--> 239         stop = np.any(self('batch_end', not self.state_dict['train']))
    240         if self.state_dict['train']:
    241             self.state_dict['iteration'] += 1

/mnt/963GB/Data/Python/Courses/fastai/fastai/fastai/callback.py in __call__(self, cb_name, call_mets, **kwargs)
    185     def __call__(self, cb_name, call_mets=True, **kwargs)->None:
    186         "Call through to all of the `CallbakHandler` functions."
--> 187         if call_mets: [getattr(met, f'on_{cb_name}')(**self.state_dict, **kwargs) for met in self.metrics]
    188         return [getattr(cb, f'on_{cb_name}')(**self.state_dict, **kwargs) for cb in self.callbacks]
    189 

/mnt/963GB/Data/Python/Courses/fastai/fastai/fastai/callback.py in <listcomp>(.0)
    185     def __call__(self, cb_name, call_mets=True, **kwargs)->None:
    186         "Call through to all of the `CallbakHandler` functions."
--> 187         if call_mets: [getattr(met, f'on_{cb_name}')(**self.state_dict, **kwargs) for met in self.metrics]
    188         return [getattr(cb, f'on_{cb_name}')(**self.state_dict, **kwargs) for cb in self.callbacks]
    189 

/mnt/963GB/Data/Python/Courses/fastai/fastai/fastai/callback.py in on_batch_end(self, last_output, last_target, **kwargs)
    272         if not is_listy(last_target): last_target=[last_target]
    273         self.count += last_target[0].size(0)
--> 274         self.val += last_target[0].size(0) * self.func(last_output, *last_target).detach().cpu()
    275 
    276     def on_epoch_end(self, **kwargs):

/mnt/963GB/Data/Python/Courses/fastai/fastai/fastai/metrics.py in exp_rmspe(pred, targ)
     45 def exp_rmspe(pred:FloatTensor, targ:FloatTensor)->Rank0Tensor:
     46     "Exp RMSE between `pred` and `targ`."
---> 47     assert pred.numel() == targ.numel(), "Expected same numbers of elements in pred & targ"
     48     if len(pred.shape)==2: pred=pred.squeeze(1)
     49     pred, targ = torch.exp(pred), torch.exp(targ)

AssertionError: Expected same numbers of elements in pred & targ
</pre>

In [ ]:
learn.save('1_databunch')

In [ ]:
learn.recorder.plot_losses(last=-1)

In [ ]:
#learn.load('1');

In [ ]:
#learn.fit_one_cycle(5, 3e-4)

In [ ]:
#learn.fit_one_cycle(5, 3e-4)

(10th place in the competition was 0.108)

## Predict

In [ ]:
preds, y = learn.get_preds(DatasetType.Test)

In [ ]:
probs = np.exp(preds)

In [ ]:
#get classes
d = {}
p = {}
i=0
for indx, prob in zip(indexes, probs):
    if i==0:
        print(f'prob: {prob}, dim: {prob.shape}, indx: {indx}')
    max_idx = np.argmax(prob)
    max_val = prob[max_idx].item()
    p[indx] = max_val
    prob_c = classes[max_idx]
    d[indx] = prob_c

In [ ]:
len(d)

In [ ]:
def pred_serial():
    preds = {}
    for idx, row in test_df.iterrows():
        pred = learn.predict(row)
        pred_str = pred[0].__str__()
        res = learn.predict(row)[2][1]
        preds[row[dep_var]]=res
        if int(idx) % 10000 == 0:
            print(f'pred_str: {pred_str}, pred: {res}, {dep_var}: {row[dep_var]}')
    return preds

In [ ]:
preds_d = pred_serial()